# Lab 3A: Data Procesing and Transformation with PySpark

## Tasks

1. **Introduction and Setup:**
    - Provide an overview of the lab and its objectives.
    - Import necessary libraries and initialize the Spark session.

2. **Data Loading:**
    - Load sample JSON data for customer details.
    - Load sample Parquet data for customer details.
    - Load sample CSV data for call records.

3. **Data Cleaning:**
    - Handle missing or empty values in customer data.
    - Remove invalid call records (e.g., duration <= 0).

4. **Data Transformation:**
    - Explode complex data types (e.g., phone numbers).
    - Add a region based on location.
    - Calculate the total duration of calls by caller_id.
    - Join customer data with total call durations.

5. **Working with Parquet Data:**
    - Write and read Parquet files.

6. **Complex JSON Data Handling:**
    - Handle nested JSON for detailed telecom metadata.
    - Explode nested arrays.

7. **Practice Case Study:**
    - Implement data transformation and data processing on a custom telecom dataset.
    - Add a region based on location.
    - Explode phone numbers.
    - Aggregate data to find the number of customers per plan and region.

8. **Additional Analysis:**
    - Perform any additional analysis or transformations as required.


## PySpark for Data Processing and Transformation: Reference Notes

**I. Core Concepts:**

* **Resilient Distributed Datasets (RDDs):**  The fundamental data structure in Spark. RDDs are immutable, fault-tolerant collections of elements that can be distributed across a cluster.  Operations on RDDs are lazily evaluated, meaning transformations are not executed until an action is performed.  Key characteristics include immutability, partitioning, and fault tolerance through lineage tracking.

* **DataFrames and Datasets:** Higher-level abstractions built on top of RDDs. DataFrames represent data in a tabular format with named columns, similar to SQL tables or Pandas DataFrames. Datasets provide type safety and optimization benefits, especially for Java/Scala, while being compatible with DataFrame APIs.

* **Spark SQL:** Enables querying DataFrames using SQL syntax, offering a familiar and powerful way to perform data analysis and manipulation.

* **Spark Streaming:**  Processes live data streams in near real-time.  Data streams are ingested, transformed, and analyzed as they arrive.

* **MLlib (Spark Machine Learning Library):** A comprehensive set of machine learning algorithms and utilities for building and deploying machine learning models.  Supports various tasks, including classification, regression, clustering, and dimensionality reduction.

* **GraphX:**  A library for graph processing, enabling operations on graph-structured data.


**II. Data Processing Operations:**

* **Transformations:** Operations that create new RDDs, DataFrames, or Datasets from existing ones without triggering immediate computation. Examples include `map`, `filter`, `flatMap`, `join`, `union`, `groupBy`, etc. Transformations are lazily evaluated.

* **Actions:** Operations that trigger the execution of transformations and produce a result. Examples include `count`, `collect`, `take`, `first`, `saveAsTextFile`, etc. Actions initiate the actual computation.


**III. Data Transformation Techniques:**

* **Data Cleaning:** Handling missing values, outliers, and inconsistent data. Techniques include imputation, removal, or transformation of problematic data points.

* **Data Aggregation:** Combining data to compute summary statistics. This often involves `groupBy` operations followed by aggregate functions like `sum`, `mean`, `min`, `max`, `count`, etc.

* **Data Filtering:** Selecting data based on specific criteria. This is typically achieved with `filter` operations and conditional statements.

* **Data Joining:** Combining data from multiple sources based on a common key.  Various types of joins (inner, outer, left, right) are available to handle different join scenarios.

* **Data Pivoting/Reshaping:**  Changing the structure of the data.  This may involve converting rows to columns or vice versa.

* **Feature Engineering:** Creating new features from existing data to improve the performance of machine learning models.

* **Data Type Conversion:** Transforming data to the appropriate types required by downstream operations or models.

* **Window Functions:** Performing calculations over a specific set of rows related to the current row.  Useful for calculating running totals, moving averages, or other time-series operations.

* **UDFs (User-Defined Functions):** Extending Spark’s functionality by defining custom functions to process data.  UDFs can be used in transformations and SQL queries.

**IV. Performance Optimization:**

* **Data Partitioning:**  Strategically distributing data across the cluster to optimize data locality and reduce communication overhead.

* **Caching:** Storing frequently accessed RDDs or DataFrames in memory for faster retrieval.

* **Data Serialization:** Choosing efficient serialization formats to minimize data transfer times.

* **Broadcast Variables:** Distributing read-only data to all nodes in a cluster.

* **Accumulator Variables:** Aggregating values from different nodes in a cluster.



**V. Integration with Other Systems:**

* PySpark supports seamless integration with various data sources, including HDFS, AWS S3, databases, and other distributed storage systems.  Connectors are readily available for many popular databases.


#PySpark Data Processing and Transformation in Telecom Domain


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, regexp_replace, when, split, lit, struct, from_json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, MapType

#Initialize Spark Session


In [ ]:
spark = SparkSession.builder.appName("Telecom Data Processing").getOrCreate()

#Loading Data


In [ ]:
#Sample JSON Data (Customer Details)

customer_data = [
    '{"customer_id": "C001", "name": "Alice", "location": "NY", "phone_numbers": ["1234567890", "0987654321"]}',
    '{"customer_id": "C002", "name": "Bob", "location": "CA", "phone_numbers": ["1234509876"]}',
    '{"customer_id": "C003", "name": "Charlie", "location": "TX", "phone_numbers": []}',
    '{"customer_id": "C004", "name": "David", "location": "FL", "phone_numbers": ["9876543210"]}',
    '{"customer_id": "C005", "name": "Eve", "location": "NY", "phone_numbers": ["1122334455", "5566778899"]}',
    '{"customer_id": "C006", "name": "Frank", "location": "CA", "phone_numbers": []}',
    '{"customer_id": "C007", "name": "Grace", "location": "TX", "phone_numbers": ["9988776655"]}',
    '{"customer_id": "C008", "name": "Hank", "location": "NY", "phone_numbers": ["2233445566"]}',
    '{"customer_id": "C009", "name": "Ivy", "location": "CA", "phone_numbers": ["3344556677"]}',
    '{"customer_id": "C010", "name": "Jack", "location": "TX", "phone_numbers": ["4455667788", "7788990011"]}'
]

customer_schema = StructType([
    StructField("customer_id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("location", StringType(), True),
    StructField("phone_numbers", ArrayType(StringType()), True)
])

customer_df = spark.read.json(spark.sparkContext.parallelize(customer_data), schema=customer_schema)
customer_df.show(truncate=False)

In [ ]:
#Sample Parquet Data (Customer Details)
customer_df.write.mode("overwrite").parquet("customer_data.parquet")
customer_parquet_df = spark.read.parquet("customer_data.parquet")
customer_parquet_df.show(truncate=False)

In [ ]:
# #### Sample CSV Data (Call Records)
call_records_data = [
    "caller_id,receiver_id,duration,status",
    "C001,C002,5,Connected",
    "C002,C003,15,Connected",
    "C003,C001,7,Dropped",
    "C001,C003,10,Connected",
    "C002,C001,0,Failed",
    "C004,C005,12,Connected",
    "C005,C006,20,Connected",
    "C006,C007,3,Dropped",
    "C007,C008,25,Connected",
    "C008,C009,8,Connected",
    "C009,C010,30,Connected",
    "C010,C001,14,Connected",
    "C002,C004,0,Failed",
    "C003,C005,18,Connected",
    "C006,C008,6,Connected"
]

In [ ]:
call_records_rdd = spark.sparkContext.parallelize(call_records_data)
call_records_df = spark.read.option("header", True).csv(call_records_rdd)
call_records_df = call_records_df.withColumn("duration", col("duration").cast(IntegerType()))
call_records_df.show()

#Data Cleaning


In [ ]:
#Handling Missing or Empty Values
customer_df_cleaned = customer_df.withColumn(
    "phone_numbers", when(col("phone_numbers").isNull() | (col("phone_numbers").isNull()) | (col("phone_numbers")==lit("")), lit(None)).otherwise(col("phone_numbers"))
)
customer_df_cleaned.show(truncate=False)

customer_parquet_df_cleaned = customer_parquet_df.withColumn(
    "phone_numbers", when(col("phone_numbers").isNull() | (col("phone_numbers").isNull()) | (col("phone_numbers")==lit("")), lit(None)).otherwise(col("phone_numbers"))
)
customer_parquet_df_cleaned.show(truncate=False)

In [ ]:
#Removing Invalid Call Records (e.g., duration <= 0)
valid_calls_df = call_records_df.filter(col("duration") > 0)
valid_calls_df.show()

#Data Transformation


In [ ]:
#Exploding Complex Data Types
exploded_customer_df = customer_df_cleaned.withColumn("phone_number", explode(col("phone_numbers")))
exploded_customer_df.show(truncate=False)

exploded_customer_parquet_df = customer_parquet_df_cleaned.withColumn("phone_number", explode(col("phone_numbers")))
exploded_customer_parquet_df.show(truncate=False)

In [ ]:
# Adding a region based on location.
region_mapping_expr = when(col("location") == "NY", "North-East").when(col("location") == "CA", "West").when(col("location") == "TX", "South").when(col("location") == "FL", "South-East").otherwise("Unknown")

enriched_customer_df = customer_df_cleaned.withColumn("region", region_mapping_expr)
enriched_customer_df.show(truncate=False)

enriched_customer_parquet_df = customer_parquet_df_cleaned.withColumn("region", region_mapping_expr)
enriched_customer_parquet_df.show(truncate=False)

In [ ]:
# Total duration of calls by caller_id.
total_call_duration_df = valid_calls_df.groupBy("caller_id").sum("duration").withColumnRenamed("sum(duration)", "total_duration")
total_call_duration_df.show()

In [ ]:
# Joining customer data with total call durations.
final_df = enriched_customer_df.join(total_call_duration_df, enriched_customer_df.customer_id == total_call_duration_df.caller_id, "left_outer").drop("caller_id")
final_df = final_df.withColumn("total_duration", when(col("total_duration").isNull(), lit(0)).otherwise(col("total_duration")))
final_df.show(truncate=False)

final_parquet_df = enriched_customer_parquet_df.join(total_call_duration_df, enriched_customer_parquet_df.customer_id == total_call_duration_df.caller_id, "left_outer").drop("caller_id")
final_parquet_df = final_parquet_df.withColumn("total_duration", when(col("total_duration").isNull(), lit(0)).otherwise(col("total_duration")))
final_parquet_df.show(truncate=False)

#Working with Parquet Data


In [ ]:
# Writing and Reading Parquet Files
final_df.write.mode("overwrite").parquet("telecom_data_final.parquet")
final_parquet_df.write.mode("overwrite").parquet("telecom_data_final_parquet.parquet")

parquet_df = spark.read.parquet("telecom_data_final.parquet")
parquet_df.show(truncate=False)

parquet_df2 = spark.read.parquet("telecom_data_final_parquet.parquet")
parquet_df2.show(truncate=False)

#Complex JSON Data Handling


In [ ]:
# Handling Nested JSON for Detailed Telecom Metadata
nested_json_data = [
    '{"network_id": "N001", "details": {"type": "4G", "bandwidth": "100Mbps"}, "locations": ["NY", "CA"]}',
    '{"network_id": "N002", "details": {"type": "5G", "bandwidth": "1Gbps"}, "locations": ["TX", "FL"]}',
    '{"network_id": "N003", "details": {"type": "3G", "bandwidth": "20Mbps"}, "locations": ["CA", "NY"]}'
]

In [ ]:
nested_schema = StructType([
    StructField("network_id", StringType(), True),
    StructField("details", StructType([
        StructField("type", StringType(), True),
        StructField("bandwidth", StringType(), True)
    ]), True),
    StructField("locations", ArrayType(StringType()), True)
])

nested_json_df = spark.read.json(spark.sparkContext.parallelize(nested_json_data), schema=nested_schema)
nested_json_df.show(truncate=False)

In [ ]:
# Exploding Nested Arrays
exploded_network_df = nested_json_df.withColumn("location", explode(col("locations"))).drop("locations")
exploded_network_df.show(truncate=False)

#Practice Case study - Implementing data transformation and data processing on custom telecom dataset



```
#customer_data = [
    '{"customer_id": "C001", "name": "Alice", "location": "NY", "phone_numbers": ["1234567890", "0987654321"], "age": 30, "plan": "Premium"}',
    '{"customer_id": "C002", "name": "Bob", "location": "CA", "phone_numbers": ["1234509876"], "age": 25, "plan": "Basic"}',
    '{"customer_id": "C003", "name": "Charlie", "location": "TX", "phone_numbers": [], "age": 40, "plan": "Premium"}',
    '{"customer_id": "C004", "name": "David", "location": "FL", "phone_numbers": ["9876543210"], "age": null, "plan": "Basic"}', # Example of null value
    '{"customer_id": "C005", "name": "Eve", "location": "NY", "phone_numbers": ["1122334455", "5566778899"], "age": 35, "plan": "Premium"}',
    '{"customer_id": "C006", "name": "Frank", "location": "CA", "phone_numbers": [], "age": 28, "plan":"Basic"}'
]
```



In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, regexp_replace, when, split, lit, struct, from_json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, MapType
import json

# Initialize Spark Session
spark = SparkSession.builder.appName("Telecom Data Processing").getOrCreate()

# Define the schema using imported StructType and other types
customer_schema = StructType([
    StructField("customer_id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("location", StringType(), True),
    StructField("phone_numbers", ArrayType(StringType()), True),
    StructField("age", IntegerType(), True),
    StructField("plan", StringType(), True)
])

# Define the data
customer_data_processed = [ # Assign the data to the customer_data_processed variable
    {"customer_id": "C001", "name": "Alice", "location": "NY", "phone_numbers": ["1234567890", "0987654321"], "age": 30, "plan": "Premium"},
    {"customer_id": "C002", "name": "Bob", "location": "CA", "phone_numbers": ["1234509876"], "age": 25, "plan": "Basic"},
    {"customer_id": "C003", "name": "Charlie", "location": "TX", "phone_numbers": [], "age": 40, "plan": "Premium"},
    {"customer_id": "C004", "name": "David", "location": "FL", "phone_numbers": ["9876543210"], "age": None, "plan": "Basic"},  # Example of null value
    {"customer_id": "C005", "name": "Eve", "location": "NY", "phone_numbers": ["1122334455", "5566778899"], "age": 35, "plan": "Premium"},
    {"customer_id": "C006", "name": "Frank", "location": "CA", "phone_numbers": [], "age": 28, "plan": "Basic"}
]

customer_df = spark.createDataFrame(customer_data_processed, schema=customer_schema)

In [ ]:
# Data Transformation: Adding a region based on location
region_mapping_expr = when(col("location") == "NY", "North-East") \
                      .when(col("location") == "CA", "West") \
                      .when(col("location") == "TX", "South") \
                      .when(col("location") == "FL", "South-East") \
                      .otherwise("Unknown")

customer_df = customer_df.withColumn("region", region_mapping_expr)

customer_df.show(truncate=False)

+-----------+-------+--------+------------------------+----+-------+----------+
|customer_id|name   |location|phone_numbers           |age |plan   |region    |
+-----------+-------+--------+------------------------+----+-------+----------+
|C001       |Alice  |NY      |[1234567890, 0987654321]|30  |Premium|North-East|
|C002       |Bob    |CA      |[1234509876]            |25  |Basic  |West      |
|C003       |Charlie|TX      |[]                      |40  |Premium|South     |
|C004       |David  |FL      |[9876543210]            |NULL|Basic  |South-East|
|C005       |Eve    |NY      |[1122334455, 5566778899]|35  |Premium|North-East|
|C006       |Frank  |CA      |[]                      |28  |Basic  |West      |
+-----------+-------+--------+------------------------+----+-------+----------+



In [ ]:
# Data Transformation: Explode phone numbers
exploded_customer_df = customer_df.withColumn("phone_number", explode(col("phone_numbers")))
exploded_customer_df.show(truncate=False)

+-----------+-----+--------+------------------------+----+-------+----------+------------+
|customer_id|name |location|phone_numbers           |age |plan   |region    |phone_number|
+-----------+-----+--------+------------------------+----+-------+----------+------------+
|C001       |Alice|NY      |[1234567890, 0987654321]|30  |Premium|North-East|1234567890  |
|C001       |Alice|NY      |[1234567890, 0987654321]|30  |Premium|North-East|0987654321  |
|C002       |Bob  |CA      |[1234509876]            |25  |Basic  |West      |1234509876  |
|C004       |David|FL      |[9876543210]            |NULL|Basic  |South-East|9876543210  |
|C005       |Eve  |NY      |[1122334455, 5566778899]|35  |Premium|North-East|1122334455  |
|C005       |Eve  |NY      |[1122334455, 5566778899]|35  |Premium|North-East|5566778899  |
+-----------+-----+--------+------------------------+----+-------+----------+------------+



In [ ]:
# Data Aggregation: Number of customers per plan and region
customer_plan_region_df = customer_df.groupBy("plan", "region").count()
customer_plan_region_df.show()

+-------+----------+-----+
|   plan|    region|count|
+-------+----------+-----+
|Premium|North-East|    2|
|Premium|     South|    1|
|  Basic|      West|    2|
|  Basic|South-East|    1|
+-------+----------+-----+

